In [7]:
import time
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn
from models.experimental import attempt_load
from utils.general import non_max_suppression 
import telepot
from datetime import datetime
import datetime
from utils.plots import Annotator, colors, save_one_box

token = '5477504802:AAHFxyA2ZmVERFwex__eAXhESlJSY7uuxWg' # telegram token
receiver_id = 5031347294 # https://api.telegram.org/bot <TOKEN>/getUpdates
bot = telepot.Bot(token)
bot.sendMessage(receiver_id, 'Your camera is active now. ') # send a message on telegram

camera = 0 # webcam
path = 'D:/scut/scut year +scholarship app/year 3 sem 2/software engineer/hss/yolov5-master/' #path to yolov5
weights = f'{path}best.pt' #path to 'model.pt'
device = torch.device('cpu')

model = attempt_load(weights, device=device) # load FP32 model
stride = int(model.stride.max()) # model stride
cudnn.benchmark = True

# Capture with opencv and detect object
cap = cv2.VideoCapture(camera)
width, height = (640 , 480) #quality
cap.set(3, width) #width
cap.set(4, height) #height

#define date and time for comparison
now = datetime.datetime.now()
today_10am = now.replace(hour=10, minute=0, second=0, microsecond=0)
today_3pm = now.replace(hour=15, minute=0, second=0, microsecond=0)


#open camera, perform inference and check conditions
while(cap.isOpened()):
    time.sleep(0.2)
    ret, frame_ = cap.read()
    frame = cv2.resize(frame_, (width, height), interpolation = cv2.INTER_AREA)
    
    #display the current time on the stream
    font = cv2.FONT_HERSHEY_PLAIN
    frame = cv2.putText(frame,str(datetime.datetime.now()),(10,30), font, 1.5,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow("frame", frame)
    if cv2.waitKey(25) & 0xFF == ord('q'): #press q to exit the stream 
        break
        
    #inference
    if ret ==True:
        img = torch.from_numpy(frame).float().to(device).permute(2,0,1)
        img/=255.0
        
        if img.ndimension() ==3:
            img = img.unsqueeze(0)
            
        results = model(img, augment=False)[0]                
        pred = non_max_suppression(results, 0.10, 0.20, agnostic=True)
                    
        #loop and pass pred in conditions
        for det in pred:
            if len(det):
                conf_, class_ = det[0][4], int(det[0][5])
                
                x1 = int(pred[0][0][0].item())
                y1 = int(pred[0][0][1].item())
                x2 = int(pred[0][0][2].item())
                y2 = int(pred[0][0][3].item())
                
                #fire and smoke
                if (class_ ==0 or class_==1) and conf_> 0.10:
                    time_stamp = int(time.time())
                    fcm_photo = f'{path}/detected/{time_stamp}.png'
                    
                    bgr = (0, 0, 255) # color of the box
                    image = cv2.rectangle(frame,(x1, y1), (x2, y2),bgr, 3) #Plot the boxes
                    #add text
                    cv2.putText(image, str(model.names[class_]), (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
                    cv2.imwrite(fcm_photo, image)
                    
                    bot.sendPhoto(receiver_id, photo=open(fcm_photo, 'rb'))
                    bot.sendMessage(receiver_id, 'ATTENTION! \nFire or Smoke has been detected at '+str(now))
                    print(f'{time_stamp}.png has been sent.fire')
                    time.sleep(1)
                    
                #person 
                if (now > today_10am) and (now < today_3pm) and (class_==2) and conf_> 0.10:
                        
                    time_stamp = int(time.time())
                    fcm_photo = f'{path}/detected/{time_stamp}.png'
                    
                    bgr = (0, 0, 255) # color of the box
                    image = cv2.rectangle(frame,(x1, y1), (x2, y2),bgr, 3) #Plot the boxes
                    #add text
                    cv2.putText(image, str(model.names[class_]) , (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
                    cv2.imwrite(fcm_photo, image)
                    
                    
                    bot.sendPhoto(receiver_id, photo=open(fcm_photo, 'rb'))
                    bot.sendMessage(receiver_id, 'ATTENTION! \nA person has been detected at '+str(now))
                    print(f'{time_stamp}.png has been sent.human')
                    time.sleep(1)
        
    else:
        break

cap.release()
cv2.destroyAllWindows()

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs


1655349586.png has been sent.human
1655349590.png has been sent.human
1655349593.png has been sent.human
1655349597.png has been sent.human
